In [ ]:
### Step1. 기존에 저장 된 벡터 인덱스 불러오기(없을 경우 새로 생성하여 사용 필요 함)
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./index/ch03_vector_index_storage") 
vector_index = load_index_from_storage(storage_context)

Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_vector_index_storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_vector_index_storage\index_store.json.


In [ ]:
### LLM 설정
## API KEY 설정
from dotenv import load_dotenv
load_dotenv()

## 모델 설정
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(model="gpt-4o", temperature=0)  # 모델명은 예시

In [ ]:
### Step2. 다중 쿼리 생성 사용자 정의 클래스 MultiQueryTransform 작성
from llama_index.core import QueryBundle
from llama_index.core.indices.query.query_transform.base import DecomposeQueryTransform
from typing import Optional, Dict, List
from llama_index.core.prompts import PromptTemplate
from llama_index.core.llms import LLM

class MultiQueryTransform(DecomposeQueryTransform):
    """
    MultiQueryTransform
    `DecomposeQueryTransform`를 상속하여, 하나의 쿼리를 여러 개의 서브 쿼리로 분해하는 기능을 제공합니다.
    `sub_query_num`이 몇 개인지에 따라 LLM 프롬프트를 통해 여러 개의 쿼리를 생성합니다.
    생성된 서브 쿼리는 최종적으로 QueryBundle의 `custom_embedding_strs`에 추가됩니다.

    Args:
        sub_query_num (int): 분해될 쿼리 수
        llm (Optional[LLM]): LLM 객체
        verbose (bool): 중간 변환 과정을 출력할지 여부
    """

    def __init__(
        self,
        sub_query_num: int,
        llm: Optional[LLM] = None,
        verbose: bool = False,
    ) -> None:
        super().__init__(llm=llm, verbose=verbose)
        self.sub_query_num = sub_query_num

        # 멀티 쿼리 프롬프트 정의
        # {query_str}와 {sub_query_num}은 LLM에 전달될 때 실제 값으로 치환됩니다.
        self._multi_query_prompt: PromptTemplate = PromptTemplate(
            template=(
                "다음 [Original Query]와 관련 있는 쿼리를 {sub_query_num}가지로 변환시켜주세요.\n"
                "출력 포맷은 아래와 같은 형태를 지켜주세요. 변환 쿼리 앞에 #을 붙여주세요.\n\n"
                "Original Query: {query_str}\n"
                "Sub-Queries:\n"
                "#변환된 쿼리\n"
            )
        )

    def _run(self, query_bundle: QueryBundle, metadata: Dict) -> QueryBundle:
        """Run query transform."""
        # 기존 쿼리
        query_str = query_bundle.query_str

        # LLM에 전달하여 여러 개의 서브 쿼리를 생성
        llm_output = self._llm.predict(
            prompt=self._multi_query_prompt,
            query_str=query_str,
            sub_query_num=self.sub_query_num,
        )

        if self.verbose:
            print(f"=== Original Query ===\n{query_str}\n")
            print(f"=== LLM Output ===\n{llm_output}\n")

        # 최종적으로 QueryBundle을 생성하되,
        # original query_str를 그대로 유지하고, sub_queries를 custom_embedding_strs에 담아 반환
        return QueryBundle(
            query_str=query_str,  # 원본 쿼리는 그대로 유지
            custom_embedding_strs=self._parsing_llm_output(llm_output=llm_output),  # 분해된 쿼리를 추가
        )
    # 쿼리 분해 결과를 파싱하는 메서드
    def _parsing_llm_output(self, llm_output)->List[str]:
        # LLM으로부터 받은 결과를 파싱하여 서브 쿼리만 추출
        # 예) Sub-Queries:
        #     #첫번째 쿼리
        #     #두번째 쿼리
        sub_queries = []
        for line in llm_output.splitlines():
            line = line.strip()
            if line.startswith("#"):
                # '#' 기호를 제외한 문자열을 서브 쿼리로 간주
                sub_query = line.lstrip("#").strip()
                if sub_query:
                    sub_queries.append(sub_query)
        if self.verbose:
            print(f"=== Parsed Sub-Queries ===\n{sub_queries}\n")
        return sub_queries
        

In [6]:
decompose = MultiQueryTransform(sub_query_num=3)
query = "기펜재가 뭐에요?"
query_bundle = decompose.run(query)

In [7]:
print(f'원본 쿼리 : {query_bundle.query_str}')
print(f'변환 쿼리 : {query_bundle.custom_embedding_strs}')

원본 쿼리 : 기펜재가 뭐에요?
변환 쿼리 : ['기펜재의 정의는 무엇인가요?', '기펜재의 예시를 알려주세요.', '기펜재와 일반 재화의 차이점은 무엇인가요?']


In [ ]:
### Step3. 다중 쿼리 RAG 쿼리 엔진 생성
query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.query_engine import TransformQueryEngine
multi_query_engine = TransformQueryEngine(query_engine=query_engine,
                                         query_transform=decompose)
response = multi_query_engine.query(query_bundle)
print(response)

기펜재는 일반적인 수요의 법칙과는 반대로, 가격이 하락할 때 오히려 수요량이 감소하는 재화를 말합니다. 이는 수요의 법칙에 위배되는 현상으로, 이러한 재화를 처음 관찰한 학자의 이름을 따서 기펜재라고 부릅니다. 기펜재는 열등재의 일종으로, 소득이 증가함에 따라 수요가 감소하는 특성을 가지고 있습니다.


In [ ]:
# 답변 생성에 참고한 노드 확인
response.source_nodes

[NodeWithScore(node=TextNode(id_='4a3546e8-6ba7-4d93-9fe5-31d46f42c1c9', embedding=None, metadata={'page_label': '72', 'file_name': '경제금융용어_700선_sample.pdf', 'file_path': 'c:\\Users\\USER\\Desktop\\Project\\llamaindex_practice\\data\\경제금융용어_700선_sample.pdf', 'file_type': 'application/pdf', 'file_size': 1358899, 'creation_date': '2025-01-01', 'last_modified_date': '2025-01-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0ff9e1bd-ff7d-48e0-a071-53bd986bb219', node_type='4', metadata={'page_label': '72', 'file_name': '경제금융용어_700선_sample.pdf', 'file_path': 'c:\\Users\\USER\\Desktop\\Project\\llamaindex_practice\\data\\경제금융용어_700선_sample.pdf', 'file_type': 'application/pdf', 'file_size': 1358899, 'creation